In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import max_functions as mf

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS


path = r'D:\UniFRK\SEXTANT_sept2018\S-TFMoX\enant=S_hel=+1_KE=3.0eV' # use your path
all_files = glob.glob(path + "/*.dat")

li = []
colnames=["phi","cos(theta)","value"] 

# how to load multiple files http://jonathansoma.com/lede/foundations-2017/classes/working-with-many-files/class/
for filename in all_files:
    df = pd.read_csv(filename, delimiter=r"\s+", names=colnames, header=None) 
    # or delim_whitespace=True, it is faste
    # r"\s+" is a regex (regular expression)
    df["filename"]=filename.split("\\")[-1].split(".")[0] # adding a column with the file name
    df["filename"]=df["filename"].str.replace(" ","") # corrects for spaces
    df["filename"]=df["filename"].astype("category")
    li.append(df) # a unique DataFrame

frame = pd.concat(li, axis=0)
frame_srt=frame.groupby("filename")
frame.loc[frame["filename"] == "1_1"] # first way of selecting the right file
frame_set = frame.set_index("filename") # build a multiindex using the categories of filename

phi = frame.loc[frame["filename"] == "1_1"].iloc[:,0].to_numpy() # phi
ctheta = frame.loc[frame["filename"] == "1_1"].iloc[:,1].to_numpy() # cos(theta)
counts = frame.loc[frame["filename"] == "1_1"].iloc[:,2].to_numpy() # counts

theta_rad = np.arccos(ctheta)
phi_rad = phi * np.pi/180.

# convertion to cartesian coordinates 1D vectors of shape (2000,1)
x = counts * np.sin(theta_rad) * np.cos(phi_rad)
y = counts * np.sin(theta_rad) * np.sin(phi_rad)
z = counts * ctheta

# distance from the origin for the cmap
# remember do not use math, but np for fast operations on vectors
d = np.sqrt(x**2+y**2+z**2)

# reshaping to have the right dimension for 3D
X = np.reshape(x, (200, 100))
Y = np.reshape(y, (200, 100))
Z = np.reshape(z, (200, 100))

d_matrix = np.sqrt(X**2+Y**2+Z**2)

In [ ]:
# fig = plt.figure(figsize=plt.figaspect(0.15))
# alternative sintax
fig = plt.figure(figsize=(40,10))
# ax = fig.gca(projection='3d')

ax = fig.add_subplot(1, 6, 1)
# ax.scatter(x)
ax.plot(ctheta)
ax.title.set_text('ctheta axis')

ax = fig.add_subplot(1, 6, 2)
ax.plot(np.unique(ctheta))
ax.title.set_text('ctheta axis unique')

ax = fig.add_subplot(1, 6, 3)
# ax.scatter(y)
ax.plot(phi)
ax.title.set_text('phi axis')

ax = fig.add_subplot(1, 6, 4)
ax.plot(np.unique(phi))
ax.title.set_text('phi axis unique')

ax = fig.add_subplot(1, 6, 5)
# ax.scatter(z)
ax.plot(counts)
ax.title.set_text('counts axis')

ax = fig.add_subplot(1, 6, 6)
ax.plot(ctheta,phi)
ax.title.set_text('ctheta vs phi')


fig.tight_layout()
plt.show()

In [ ]:
print("phi min max", np.min(phi), np.max(phi))
print("ctetha min max", np.min(ctheta), np.max(ctheta))

In [ ]:
# fig = plt.figure(figsize=plt.figaspect(0.15))
# alternative sintax
fig = plt.figure(figsize=(40,10))
# ax = fig.gca(projection='3d')

ax = fig.add_subplot(1, 6, 1)
# ax.scatter(x)
ax.plot(x)
ax.title.set_text('x axis')

ax = fig.add_subplot(1, 6, 2)
ax.plot(np.unique(x))
ax.title.set_text('x axis unique')

ax = fig.add_subplot(1, 6, 3)
# ax.scatter(y)
ax.plot(y)
ax.title.set_text('y axis')

ax = fig.add_subplot(1, 6, 4)
ax.plot(np.unique(y))
ax.title.set_text('y axis unique')

ax = fig.add_subplot(1, 6, 5)
# ax.scatter(z)
ax.plot(z)
ax.title.set_text('z axis')

ax = fig.add_subplot(1, 6, 6)
ax.plot(x,y)
ax.title.set_text('x vs y')


fig.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=plt.figaspect(0.25))
# alternative sintax
# fig = plt.figure()
# ax = fig.gca(projection='3d')

ax = fig.add_subplot(1, 4, 1, projection='3d')
ax.scatter(x, y, z, c=d, cmap='viridis', alpha=0.2) # X,Y,Z 1D vectors -> everything ok
ax.title.set_text('scattered with c colour')

ax = fig.add_subplot(1, 4, 2, projection='3d')
ax.plot_wireframe(X,Y,Z)
ax.title.set_text('wired')

ax = fig.add_subplot(1, 4, 3, projection='3d')
verts = [list(zip(x, y, z))]
ax.plot_trisurf(x, y, z, triangles=verts, antialiased=True)
ax.title.set_text('triagnulated')

ax = fig.add_subplot(1, 4, 4, projection='3d')
ax.plot(x, y, z, antialiased=True)
ax.title.set_text('Simple plot')

fig.tight_layout()
plt.show()

In [ ]:
fig = plt.figure()
fig, ax = plt.subplots(1,2, subplot_kw={"projection": "3d"}, figsize=(20,10))

ax = fig.add_subplot(1, 2, 1, projection='3d')
ax.plot_surface(X,Y,Z)
ax.title.set_text('surface (broken on the side)')


ax = fig.add_subplot(1, 2, 2, projection='3d')
cs = ax.contour(X,Y,d_matrix, levels=30)
cs.changed()
ax.title.set_text('contour in 3D')

plt.show()

In [ ]:
Xt = np.reshape(x, (100, 200))
Yt = np.reshape(y, (100, 200))
Zt = np.reshape(z, (100, 200))

fig = plt.figure()
fig, ax = plt.subplots(1,2, subplot_kw={"projection": "3d"}, figsize=(20,10))

ax = fig.add_subplot(1, 2, 1, projection='3d')
ax.plot_surface(Xt,Yt,Zt)
ax.title.set_text('surface transposed')

ax = fig.add_subplot(1, 2, 2, projection='3d')
cs = ax.contour(Xt,Yt,d_matrix.T, levels=30)
cs.changed()
ax.title.set_text('contour transposed')


plt.show()

In [ ]:
fig = plt.figure()
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

ax.scatter(x, y, z, c=d, cmap='viridis', alpha=0.4) # X,Y,Z 1D vectors -> everything ok

# the right sostitution is the ax.plot or ax.fill_between (apparently non supported by 3daxes)
ax.contourf(X, Y, Z, zdir='z', alpha=0.2, offset=z.min())
ax.contourf(X, Y, Z, zdir='y', alpha=0.2, offset=y.max())
ax.contourf(X, Y, Z, zdir='x', alpha=0.2, offset=x.min())

ax.set_xlabel('x')
ax.set_xlim3d(x.min(), x.max())
ax.set_ylabel('y')
ax.set_ylim3d(y.min(), y.max())
ax.set_zlabel('z')
ax.set_zlim3d(z.min(), z.max())
ax.title.set_text("scattered with contour projections alpha=0.44")

plt.show()


# reminder for the sintax of ax.plot
ax.plot(x, y, 'o', ms=5, alpha=0.1, mfc='blue', mec="blue", zdir='z', zs=z_str.min())

In [ ]:
fig = plt.figure()
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

ax.scatter(x, y, z, c=d, cmap='viridis', alpha=0.5) # X,Y,Z 1D vectors -> everything ok

# the right sostitution is the ax.plot or ax.fill_between (apparently non supported by 3daxes)
ax.contourf(X, Y, Z, zdir='z', alpha=0.4, offset=z.min())
ax.contourf(X, Y, Z, zdir='y', alpha=0.4, offset=y.max())
ax.contourf(X, Y, Z, zdir='x', alpha=0.4, offset=x.min())

ax.set_xlabel('x')
ax.set_xlim3d(x.min(), x.max())
ax.set_ylabel('y')
ax.set_ylim3d(y.min(), y.max())
ax.set_zlabel('z')
ax.set_zlim3d(z.min(), z.max())
ax.title.set_text("scattered with contour projections alpha=0.5")

plt.show()